In [171]:
import os
import folium
import pandas as pd
import networkx as nx
from folium.map import *
from folium import plugins
from folium.plugins import FloatImage
from tqdm import tnrange, tqdm_notebook
from folium.plugins import MeasureControl

In [2]:
files_names = os.listdir('./Files')

In [3]:
files_names

['USA-road-d.CAL.co', 'USA-road-d.CAL.gr', 'USA-road-t.CAL.gr']

In [4]:
os.getcwd() + '\\Files\\' + files_names[0]

'C:\\Users\\Giorgio\\Documents\\Data Science\\ADM\\HW5\\Files\\USA-road-d.CAL.co'

In [5]:
dataframes_names = ['coordinates', 'physical_dist', 'time_dist']

In [111]:
for k in tnrange(3):
    if os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[0]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[0], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "ID_Node", "Longitude", "Latitude"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)
    elif os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[1]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[1], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "Node_1", "Node_2", "Physical_distance"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)
    elif os.getcwd() + '\\Files\\' + files_names[k] == os.getcwd() + '\\Files\\' + files_names[2]:
        globals()[dataframes_names[k]] = pd.read_csv(os.getcwd() + '\\Files\\' + files_names[2], skiprows = 7, sep = " ", 
                                          delimiter = " ", names = ["Character", "Node_1", "Node_2", "Time_distance"],
                                          index_col = None, usecols = None, encoding = 'ISO-8859-1')
        eval(dataframes_names[k]).drop(columns = ["Character"], inplace = True)

In [7]:
coordinates.head()

,ID_Node,Longitude,Latitude
0,1,-114315309,34133550
1,2,-114223946,34176221
2,3,-114307299,34148791
3,4,-114318765,34138889
4,5,-114347300,34042614


In [8]:
physical_dist.head()

,Node_1,Node_2,Physical_distance
0,1,1048577,456
1,1048577,1,456
2,2,1048578,2389
3,1048578,2,2389
4,3,1048579,358


In [9]:
time_dist.head()

,Node_1,Node_2,Time_distance
0,1,1048577,1139
1,1048577,1,1139
2,2,1048578,5972
3,1048578,2,5972
4,3,1048579,895


In [10]:
complete = pd.merge(physical_dist, time_dist, on = ['Node_1', 'Node_2'])

In [11]:
complete

,Node_1,Node_2,Physical_distance,Time_distance
0,1,1048577,456,1139
1,1048577,1,456,1139
2,2,1048578,2389,5972
3,1048578,2,2389,5972
4,3,1048579,358,895
...,...,...,...,...
4712813,1890815,1048576,5642,14105
4712814,989247,1048576,3212,8030
4712815,1048576,989247,3212,8030
4712816,1048572,1890810,7230,18076


In [172]:
G = nx.from_pandas_edgelist(complete, 'Node_1', 'Node_2', ['Physical_distance', 'Time_distance'], create_using = nx.DiGraph())

In [173]:
G[1]

AtlasView({1048577: {'Physical_distance': 456, 'Time_distance': 1139}, 1803: {'Physical_distance': 626, 'Time_distance': 1565}})

In [174]:
nx.is_directed(G)

True

In [179]:
def dijkstra(graph, source, destination, measure = 'network_distance'):
    len_graph = len(graph.nodes())
    distance = [float('inf')] * len_graph
    distance[source] = 0
    shortest_path = [False] * len_graph
    routes = {source: [source]}
    queue = {source: 0}
    while distance[destination] == float('inf') or shortest_path[destination] == False:        
        m_d = min_distance(len_graph, distance, shortest_path, queue)
        queue.pop(m_d)
        shortest_path[m_d] = True
        neighbours = find_neighbour(graph, m_d)
        for neighbour in neighbours: 
            current_weight = float(get_weight(graph, m_d, neighbour, measure))
            if current_weight > 0 and shortest_path[neighbour] == False and distance[neighbour] > distance[m_d] + current_weight: 
                distance[neighbour] = distance[m_d] + current_weight
                routes[neighbour] = routes[m_d] + [neighbour]
                queue[neighbour] = distance[neighbour]
    return routes[destination], distance[destination]

def min_distance(length, distance, shortest_path, queue): 
    min = float('inf')
    min_index = float('inf')
    for v, val in queue.items(): 
        if shortest_path[v] == False:
            if  val < min:
                min = val 
                min_index = v 
    return min_index 


def find_neighbour(graph, node):
    return [n[1] for n in list(graph.edges(node))]


def get_weight(graph, node_a, node_b, measure = 'network'):
    if graph.get_edge_data(node_a, node_b) != None:
        if measure == 'network':
            return 1
        else:
            return graph.get_edge_data(node_a, node_b)[measure]
    else:
        return 0

def shortest_ordered_route(graph, source, destinations, measure = 'network'):
    source  = source
    total_steps = []
    total_weight = 0
    for destination in destinations:
        steps, weight = dijkstra(graph, source, destination, measure)
        total_steps += steps
        total_weight += weight
        source = destination
    print("Path:")
    print(*total_steps)
    if measure == 'network':
        print("Network distance:")
        print(total_weight)
    elif measure == 'Time_distance':
        print("Distance in minutes")
        print(total_weight)
    elif measure == 'Physical_distance':
        print("Distance in meters:")
        print(total_weight)
    return 


In [216]:
s_o_r = shortest_ordered_route(G, 1, [3, 8, 25, 1805456])

Path:
1 1048577 1766 1048579 3 3 1048579 1766 1048577 1 1803 1802 1050022 1799 1050018 1050332 2206 2205 1050348 2214 1050379 2251 1050363 2229 2237 1050369 2238 1050368 6 5 1048581 8 8 1050376 2246 1050304 1050305 2647 2157 1050306 1050308 2159 1050315 2163 1050312 1050317 2171 2170 1050316 2179 2180 2344 1050537 1050313 2167 2172 2073 1050238 1050239 1050241 1050496 2384 1048594 23 24 1048595 25 25 1048595 24 23 1048594 2384 1050496 1050241 1050239 2074 2181 1050294 2095 1050325 2183 488120 1440651 1440640 488107 488127 567776 567774 567775 1504613 567772 567767 567764 567760 1504586 567740 567752 1504594 567726 1504534 567673 1504535 567148 567147 1504112 567149 567150 1504115 1504109 567144 1504108 567139 567140 567133 1504101 567137 567138 567114 567115 567113 567112 567103 567104 567101 567102 1504076 567100 567099 1504075 567098 1504074 1503954 566951 566716 1503768 566715 1503767 1503766 566714 566713 566712 1503765 1503928 1503929 566920 1503927 566724 1503774 566725 1503777 5

In [191]:
nx.dijkstra_path(G, 1, 3)

[1, 1048577, 1766, 1048579, 3]

In [215]:
m = folium.Map(location = ((coordinates.set_index('ID_Node')['Latitude'][s_o_r[0]])/1000000, 
                           (coordinates.set_index('ID_Node')['Longitude'][s_o_r[0]])/1000000), zoom_start = 10, tiles = 'openstreetmap')
for node in s_o_r:
    folium.CircleMarker(location = ((coordinates.set_index('ID_Node')['Latitude'][node])/1000000, 
                                    (coordinates.set_index('ID_Node')['Longitude'][node])/1000000), radius = 3, line_color = '#3186cc', fill_color = '#FFFFFF', fill_opacity = 0.7, fill = True).add_to(m)
    
display(m)